In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob, os
import shutil

In [2]:
df = pd.read_pickle('timex_inventory.pkl')
df.sample(5)

,dts,dto,fnamec1,datenum,doy,tide,hsn,az,zen,avg1,...,red1,green1,blue1,red2,green2,blue2,sharp1,sharp2,contrast1,contrast2
517,2021-03-30T14:30:00,2021-03-30 14:30:00,1617114600.c1.timex.jpg,738245.6042,89.0,0.0380,-2.5718,125.9896,51.9094,130.1010,...,121.1897,128.1110,141.0023,122.4078,128.0594,136.6277,0.0000,0.0000,0.0000,0.0000
1045,2021-04-28T12:00:00,2021-04-28 12:00:00,1619611200.c1.timex.jpg,738274.5000,118.0,-1.5088,-5.0718,88.7531,69.7358,130.2267,...,118.8102,127.6448,144.2251,114.9804,122.2601,132.7236,0.0000,0.0002,0.0000,0.0509
94,2021-03-14T11:00:00,2021-03-14 11:00:00,1615719600.c1.timex.jpg,738229.4583,73.0,-1.3068,-6.0718,89.4929,94.0737,133.2057,...,144.8151,129.9126,124.8894,124.6085,114.5550,108.1071,0.0003,0.0013,0.0232,0.1015
1457,2021-05-21T22:00:00,2021-05-21 22:00:00,1621634400.c1.timex.jpg,738297.9167,141.0,0.5529,4.9282,275.8956,65.4117,109.2222,...,123.0819,107.9629,96.6218,136.1989,117.8559,104.2258,0.0000,0.0000,0.0244,0.0000
70,2021-03-13T13:00:00,2021-03-13 13:00:00,1615640400.c1.timex.jpg,738228.5417,72.0,-0.2425,-4.0718,110.5223,72.5378,112.4989,...,149.1126,109.6775,78.7066,129.5162,111.9745,94.7913,0.0000,0.0003,0.0000,0.0563


In [3]:
# remove dark images
df = df.drop(df[df['avg1']< 80].index)
df = df.drop(df[df['avg2']< 80].index)

In [4]:
dfs = df.sample(20)

In [16]:
# move the c2 images to a new directory
old_dir = 'F:/timex/'
new_dir = 'F:/timex_movie_input/'
mov_dir = 'F:/timex_movie/'
if not os.path.exists(new_dir):
    os.makedirs(new_dir)
if not os.path.exists(mov_dir):
    os.makedirs(mov_dir)

ic = 0
for index, row in df.iterrows():
    old_file = row['fnamec1'].replace('c1','c2')
    # old_file = row['fnamec1']
    old_path = old_dir+old_file
    new_name = 'c2{:04.0f}.jpg'.format(ic)
    #new_name = 'c1{:04.0f}.jpg'.format(ic)
    tstring = row['dts']
    tstring2 = 'Tide: {:03.1f} Sun: {:03.1f}'.format(row['tide'], row['hsn'])
    #print(tstring)
    #print(tstring2)
    ic = ic+1
    new_path = new_dir+new_name
    #print(old_path, new_path)

    magick_cmd = ' -gravity NorthEast -font Arial -pointsize 70 -fill white -stroke black -strokewidth 10 -annotate +40+40 "{0}" -stroke none -annotate +40+40 "{0}" -fill white -stroke black -strokewidth 10 -annotate +40+120 "{1}" -stroke none -annotate +40+120 "{1}" '.format(tstring, tstring2)
    mcmd = 'magick "{}" {} "{}"'.format(old_path, magick_cmd, new_path)
    # print(mcmd)
    os.system(mcmd)

In [18]:
# I used this...
ffmpeg_cmd = 'ffmpeg -framerate 10 -i F:/timex_movie_input/c2%04d.jpg  -q:v 3 F/timex_movie/timex.avi'
#print(ffmpeg_cmd)
# Dan uses these switches. He also uses glob, but that option is not in the conda dist of ffmpeg
ffmpeg_cmd = "ffmpeg -framerate 10 -i F:/timex_movie_input/c2%04d.jpg -c:v libx264 -pix_fmt yuv420p -vf scale=-1:1024 F:/timex_movie/CACO-02_c2timex2.mp4"
print(ffmpeg_cmd)
if os.system(ffmpeg_cmd) == 0:
    print('Success')


ffmpeg -framerate 10 -i F:/timex_movie_input/c2%04d.jpg -c:v libx264 -pix_fmt yuv420p -vf scale=-1:1024 F:/timex_movie/CACO-02_c2timex2.mp4
Success
